# 🌍 Geolocalização de Imóveis - Google Colab

**Encontra o endereço exato de um imóvel a partir de uma foto.**

## 🎯 Pipeline:
1. Análise Visual (GPT-4o) → 2. Busca Geográfica (Google) → 3. Matching (CLIP+SIFT) → 4. Validação (GPT-4o)

## ⚡ GPU Recomendada:
**Runtime → Alterar tipo → GPU T4** (acelera 4x)

---

In [ ]:
# 📤 2. UPLOAD DO PROJETO
# Opção A: Clone do repositório (se você tiver um repo Git)
# !git clone https://github.com/seu-usuario/geolocalizacao.git
# %cd geolocalizacao/geolocaliza

# Opção B: Upload manual dos arquivos
from google.colab import files
import shutil
import os

# Criar estrutura de diretórios
!mkdir -p /content/geolocaliza/agents
!mkdir -p /content/geolocaliza/output
!mkdir -p /content/geolocaliza/cache
%cd /content/geolocaliza

print("📤 Faça upload dos arquivos .py do projeto:")
print("   IMPORTANTE: Faça upload de TODOS os arquivos:")
print("   • config.py")
print("   • main.py")
print("   • agents/vision_agent.py")
print("   • agents/search_agent.py")
print("   • agents/matching_agent.py")
print("   • agents/validation_agent.py")
print("\nSelecione TODOS os arquivos de uma vez...\n")

uploaded = files.upload()

# Organizar arquivos
for filename in uploaded.keys():
    if 'agent' in filename.lower():
        # Mover para pasta agents
        dest = f'/content/geolocaliza/agents/{os.path.basename(filename)}'
        if os.path.exists(filename):
            shutil.move(filename, dest)
            print(f"✓ {filename} → agents/")
    else:
        # Mover para raiz
        dest = f'/content/geolocaliza/{os.path.basename(filename)}'
        if os.path.exists(filename) and filename != dest:
            shutil.move(filename, dest)
            print(f"✓ {filename} → raiz")

# Criar __init__.py vazio na pasta agents
with open('/content/geolocaliza/agents/__init__.py', 'w') as f:
    f.write('')

print("\n✅ Arquivos organizados!")
print("\n📁 Estrutura:")
!ls -la /content/geolocaliza/
print("\n📁 Agents:")
!ls -la /content/geolocaliza/agents/

In [ ]:
# 🔍 2.5. VERIFICAR ARQUIVOS (Execute esta célula para confirmar)
import os

print("🔍 Verificando arquivos necessários...\n")

arquivos_necessarios = [
    '/content/geolocaliza/config.py',
    '/content/geolocaliza/main.py',
    '/content/geolocaliza/agents/__init__.py',
    '/content/geolocaliza/agents/vision_agent.py',
    '/content/geolocaliza/agents/search_agent.py',
    '/content/geolocaliza/agents/matching_agent.py',
    '/content/geolocaliza/agents/validation_agent.py'
]

todos_ok = True
for arquivo in arquivos_necessarios:
    existe = os.path.exists(arquivo)
    status = "✅" if existe else "❌"
    print(f"{status} {arquivo.replace('/content/geolocaliza/', '')}")
    if not existe:
        todos_ok = False

print()
if todos_ok:
    print("🎉 Todos os arquivos estão no lugar correto!")
    print("✅ Pode prosseguir para a próxima célula.")
else:
    print("⚠️  ATENÇÃO: Alguns arquivos estão faltando!")
    print("\n💡 Solução:")
    print("   1. Volte para a célula anterior (Upload do Projeto)")
    print("   2. Faça upload de TODOS os arquivos .py")
    print("   3. Execute novamente esta célula de verificação")

In [ ]:
# 🔑 3. CONFIGURAR APIs
import os
from getpass import getpass

print("🔐 Configure suas chaves de API:\n")
print("Google Cloud: https://console.cloud.google.com/")
print("OpenAI: https://platform.openai.com/\n")

GOOGLE_KEY = getpass("Google API Key: ")
OPENAI_KEY = getpass("OpenAI API Key: ")

os.environ["GOOGLE_API_KEY"] = GOOGLE_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_KEY

with open('.env', 'w') as f:
    f.write(f"GOOGLE_API_KEY={GOOGLE_KEY}\n")
    f.write(f"OPENAI_API_KEY={OPENAI_KEY}\n")

print("\n✅ APIs configuradas!")
print("💰 Custo estimado: $3-8 por busca")

In [ ]:
# 🔑 3. CONFIGURAR APIs
import os
from getpass import getpass

print("🔐 Configure suas chaves de API:\n")
print("Google Cloud: https://console.cloud.google.com/")
print("Anthropic: https://console.anthropic.com/\n")

GOOGLE_KEY = getpass("Google API Key: ")
ANTHROPIC_KEY = getpass("Anthropic API Key: ")

os.environ["GOOGLE_API_KEY"] = GOOGLE_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_KEY

with open('.env', 'w') as f:
    f.write(f"GOOGLE_API_KEY={GOOGLE_KEY}\n")
    f.write(f"ANTHROPIC_API_KEY={ANTHROPIC_KEY}\n")

print("\n✅ APIs configuradas!")
print("💰 Custo estimado: $3-8 por busca")

In [ ]:
# 📸 4. UPLOAD DA FOTO
from google.colab import files
from PIL import Image

print("📸 Faça upload da foto do imóvel:\n")
uploaded = files.upload()
foto_path = list(uploaded.keys())[0]

img = Image.open(foto_path)
img.thumbnail((800, 800))
display(img)

print(f"\n✅ Foto: {foto_path}")

In [ ]:
# 🗺️ 5. CONFIGURAR BUSCA
# Use Google Maps para pegar coordenadas aproximadas

cidade = "São Paulo"
bairro = "Alto da Boa Vista"  # Opcional
center_lat = -23.6505
center_lon = -46.6815
radius_m = 2000  # Raio em metros

print(f"📍 Busca configurada:")
print(f"   Cidade: {cidade}")
print(f"   Bairro: {bairro}")
print(f"   Centro: ({center_lat}, {center_lon})")
print(f"   Raio: {radius_m}m")
print(f"\n🗺️  Ver: https://www.google.com/maps/@{center_lat},{center_lon},15z")

In [ ]:
# 🚀 6. EXECUTAR GEOLOCALIZAÇÃO
import sys
import os

# Garantir que estamos no diretório correto
%cd /content/geolocaliza

# Adicionar ao path
if '/content/geolocaliza' not in sys.path:
    sys.path.insert(0, '/content/geolocaliza')

# Verificar se main.py existe
if not os.path.exists('/content/geolocaliza/main.py'):
    print("❌ ERRO: main.py não encontrado!")
    print("\n📁 Arquivos disponíveis:")
    !ls -la /content/geolocaliza/
    raise FileNotFoundError("main.py não encontrado. Volte para célula 2 e faça upload.")

# Importar
try:
    from main import GeoLocalizador
    print("✅ Módulo main importado com sucesso!\n")
except ImportError as e:
    print(f"❌ Erro ao importar: {e}")
    print("\n🔍 Diagnóstico:")
    print(f"   Diretório atual: {os.getcwd()}")
    print(f"   Python path: {sys.path[:3]}")
    print("\n📁 Conteúdo de /content/geolocaliza:")
    !ls -la /content/geolocaliza/
    raise

print("🔧 Inicializando sistema...\n")
geo = GeoLocalizador()

print("="*70)
print("🚀 INICIANDO GEOLOCALIZAÇÃO")
print("="*70)
print(f"\n⏱️  Aguarde 2-5 minutos...\n")

resultado = geo.localizar_imovel(
    foto_path=foto_path,
    cidade=cidade,
    bairro=bairro,
    center_lat=center_lat,
    center_lon=center_lon,
    radius_m=radius_m
)

print("\n" + "="*70)
print("✅ CONCLUÍDO!")
print("="*70)

In [ ]:
# 📊 7. RESULTADOS
if resultado["success"]:
    print("\n🎉 IMÓVEL LOCALIZADO!\n")
    print(f"📍 Endereço: {resultado['endereco']}")
    print(f"\n🏠 Detalhes:")
    print(f"   Rua:    {resultado['rua']}")
    print(f"   Número: {resultado['numero']}")
    print(f"   Bairro: {resultado['bairro']}")
    print(f"   Cidade: {resultado['cidade']}")
    print(f"   Estado: {resultado['estado']}")
    print(f"   CEP:    {resultado['cep']}")
    print(f"\n🎯 Confiança: {resultado['confianca']:.1%}")
    print(f"\n📊 Scores:")
    print(f"   CLIP: {resultado['scores']['clip']:.3f}")
    print(f"   SIFT: {resultado['scores']['geometria']:.3f}")
    print(f"   LLM:  {resultado['scores']['llm']:.3f}")
    print(f"\n💭 Raciocínio: {resultado['reasoning']}")
    print(f"\n🗺️  Street View: {resultado['street_view_link']}")
else:
    print(f"\n❌ Erro: {resultado['error']}")
    if 'hint' in resultado:
        print(f"💡 Dica: {resultado['hint']}")

In [ ]:
# 🗺️ 8. MAPA INTERATIVO
from IPython.display import IFrame
from pathlib import Path

mapa = "/content/geolocaliza/output/mapa.html"
if Path(mapa).exists():
    display(IFrame(mapa, width=900, height=600))
else:
    print("⚠️  Mapa não gerado")

In [ ]:
# 🖼️ 9. COMPARAÇÃO VISUAL
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

if resultado["success"]:
    df = pd.read_csv("/content/geolocaliza/output/candidatos_validados.csv")
    best = df.iloc[0]
    
    img_query = Image.open(foto_path)
    sv_path = f"/content/geolocaliza/output/street_views/{best['filename']}"
    
    if Path(sv_path).exists():
        img_sv = Image.open(sv_path)
        
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))
        axes[0].imshow(img_query)
        axes[0].set_title("FOTO DO USUÁRIO", fontsize=14, fontweight='bold')
        axes[0].axis('off')
        
        axes[1].imshow(img_sv)
        axes[1].set_title(
            f"MELHOR MATCH (Street View)\n"
            f"Confiança: {best['final_confidence']:.1%}",
            fontsize=14, fontweight='bold'
        )
        axes[1].axis('off')
        plt.tight_layout()
        plt.show()

In [ ]:
# 💾 10. DOWNLOAD RESULTADOS
from google.colab import files
import shutil

shutil.make_archive('/content/resultados', 'zip', '/content/geolocaliza/output')
files.download('/content/resultados.zip')

print("✅ Download completo!")
print("\n📁 Contém:")
print("   • resultado_final.json")
print("   • analise_visual.json")
print("   • candidatos_validados.csv")
print("   • mapa.html")
print("   • street_views/")
print("   • geolocaliza.log")